In [1]:
import itertools

import numpy as np
import pandas as pd
import scanpy as sc
from tqdm import tqdm

/home/noahcohenkalafut/miniconda3/envs/gnn/lib/python3.10/site-packages/numpy/core/getlimits.py:542: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
# Example dataset
pd.read_csv('/mnt/c/Users/nck/repos/scGNN/GSE138852/GSE138852_counts.csv.gz', index_col=0, compression='gzip')

,AAACCTGGTAGAAAGG_AD5_AD6,AAACCTGGTAGCGATG_AD5_AD6,AAACCTGTCAGTCAGT_AD5_AD6,AAACCTGTCCAAACAC_AD5_AD6,AAACCTGTCCAGTATG_AD5_AD6,AAAGCAACATGGGAAC_AD5_AD6,AAAGCAAGTCGAATCT_AD5_AD6,AAAGCAAGTTTGTTGG_AD5_AD6,AAAGTAGGTAATCACC_AD5_AD6,AAAGTAGGTTCCACGG_AD5_AD6,...,TTTGGTTAGCCACGCT_AD1_AD2,TTTGGTTCAACTTGAC_AD1_AD2,TTTGGTTCAGCCTTTC_AD1_AD2,TTTGGTTCATCGGACC_AD1_AD2,TTTGGTTTCCCAGGTG_AD1_AD2,TTTGGTTTCCGTACAA_AD1_AD2,TTTGTCACAAGCCATT_AD1_AD2,TTTGTCAGTATAGGTA_AD1_AD2,TTTGTCATCCACTGGG_AD1_AD2,TTTGTCATCCGGGTGT_AD1_AD2
FO538757.2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
AP006222.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP5-857K21.4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
RP11-206L10.9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NOC2L,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ND5,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MT-CYB,0,0,1,0,0,3,0,1,2,0,...,0,0,1,0,0,0,0,1,1,2
AL592183.1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC007325.4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Load data
adata = sc.read_h5ad('/mnt/d/PsychAD/psychAD_snRNAseq_rawCounts.h5ad', backed='r')

# Stratification params
print('Performing stratification')
stratifying_cols = ['Individual_ID', 'subclass']  # Implied stratification by source and class

# Aggregate cells with scanpy
# adata.obs['strat'] = adata.obs.apply(lambda r: ' - '.join([r[col] for col in stratifying_cols]), axis=1)
# adata_pseudobulk = sc.get.aggregate(adata, by='strat', func='sum', axis='obs')  # Not implemented

# Aggregate cells manually
print('Aggregating cells')
# Manually perform stratification
pseudo_data = []; strat_names = []
unique_strat_vals = [np.unique(adata.obs[col]) for col in stratifying_cols]
total_num = np.prod([un.shape[0] for un in unique_strat_vals])
for i, stratification in enumerate(itertools.product(*unique_strat_vals)):
    # Get mask
    mask = np.ones(adata.X.shape[0], dtype=bool)
    for col, strat in zip(stratifying_cols, stratification):
        mask *= (adata.obs[col] == strat).to_numpy()
    
    # Continue if no samples found
    if mask.sum() < 1: continue

    # CLI
    print(f'{i+1}/{total_num}\t{stratification}: {mask.sum()}')

    # Add name
    name = '_'.join(stratification)
    strat_names.append(name)

    # Aggregate samples, sum for pseudobulk
    raw = adata.X[np.argwhere(mask).flatten()]
    processed = raw.sum(axis=0)
    pseudo_data.append(processed)

# Format data
pseudo_data = np.stack(pseudo_data, axis=0)

# Save to file
df = pd.DataFrame(data=pseudo_data.T, index=adata.var_names.to_numpy(), columns=strat_names)
df.to_csv('/mnt/d/PsychAD/psychAD_pseudobulk.csv.gz', compression='gzip')
# pd.read_csv('/mnt/d/PsychAD/psychAD_pseudobulk.csv.gz', index_col=0, compression='gzip')

Performing stratification
Aggregating cells
0/40338	('AMPAD_HBCC_0000000002', 'Adaptive'): 8
1/40338	('AMPAD_HBCC_0000000002', 'Astro'): 1125
2/40338	('AMPAD_HBCC_0000000002', 'EN_L2_3_IT'): 1079
3/40338	('AMPAD_HBCC_0000000002', 'EN_L3_5_IT_1'): 40
4/40338	('AMPAD_HBCC_0000000002', 'EN_L3_5_IT_2'): 68
5/40338	('AMPAD_HBCC_0000000002', 'EN_L3_5_IT_3'): 28
6/40338	('AMPAD_HBCC_0000000002', 'EN_L5_6_NP'): 6
8/40338	('AMPAD_HBCC_0000000002', 'EN_L6B'): 4
9/40338	('AMPAD_HBCC_0000000002', 'EN_L6_CT'): 9
10/40338	('AMPAD_HBCC_0000000002', 'EN_L6_IT_1'): 9
12/40338	('AMPAD_HBCC_0000000002', 'Endo'): 108
13/40338	('AMPAD_HBCC_0000000002', 'IN_ADARB2'): 100
14/40338	('AMPAD_HBCC_0000000002', 'IN_LAMP5_LHX6'): 17
15/40338	('AMPAD_HBCC_0000000002', 'IN_LAMP5_RELN'): 44
16/40338	('AMPAD_HBCC_0000000002', 'IN_PVALB'): 23
17/40338	('AMPAD_HBCC_0000000002', 'IN_PVALB_CHC'): 25
18/40338	('AMPAD_HBCC_0000000002', 'IN_SST'): 36
19/40338	('AMPAD_HBCC_0000000002', 'IN_VIP'): 309
20/40338	('AMPAD_HBCC_000